# Discovery of the Higgs Boson
____________________

In [1]:
# Imports

import scipy 
import scipy.stats
import numpy as np
import matplotlib.pyplot as plt

# plot preset
FONTSIZE = 11
DPI = 150
X_SIZE = 6
Y_SIZE = 4

plt.style.use('seaborn')
plt.rcParams['font.size'] = FONTSIZE
plt.rcParams['xtick.labelsize'] = FONTSIZE
plt.rcParams['ytick.labelsize'] = FONTSIZE
plt.rcParams['legend.fontsize'] = FONTSIZE
plt.rcParams['axes.titlesize'] = FONTSIZE
plt.rcParams['axes.labelsize'] = FONTSIZE
plt.rcParams['figure.autolayout'] = True
plt.rcParams['figure.dpi'] = 150
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['figure.figsize'] = (X_SIZE, Y_SIZE)


## Exercise 6.4
_______

We start by simulating the data for the Higgs experiment, this is based on the true measured data by the ATLAS and CMS experiments which we only have available as binned data.
The simulation uses a *seed* to ensure that we can compare our results at the end of the exercise. However, you can give it a try without the seed and see how this changes the outcome!

In [ ]:
lambda_           = 1/35. #Decay constant of exponential background
x0                = 105 # we shift the x-variable for simpler handling
gaussloc          = 125. - x0 #125GeV = Mass of the Higgs boson
gausscale         = 2. #standard deviation of the Higgs events
nbg               = 30000 #number of background events
ratio_higgs_to_bg = 200. / 30000 #ratio of number of Higgs and background events 
nhiggs            = int(ratio_higgs_to_bg * nbg) #number of Higgs events
ntotal            = nbg + nhiggs 


## Do simulation
seed = 4242
expon_randGen = scipy.stats.expon
expon_randGen.random_state=np.random.Generator(np.random.PCG64(seed))
norm_randGen = scipy.stats.norm
norm_randGen.random_state=np.random.Generator(np.random.PCG64(seed))
sim_data   = expon_randGen.rvs ( scale=1/lambda_, size= nbg ) 
gauss = norm_randGen.rvs ( loc=gaussloc, scale=gausscale, size=nhiggs ) 
sim_data   = np.append ( sim_data, gauss )
print(ntotal, " events simulated. mu =",nhiggs / ntotal)

In [ ]:
# Plot it (Note: Nothing to add in this cell)

## Parameters
binw  = 1.0 # bin width in GeV
xmax  = 155 # maximal m_yy to draw
nbins = int((xmax -x0) / binw)

## Plot it (do not forget to shift by x0)
plt.hist(sim_data+x0,bins=nbins,range=(x0,xmax))
plt.xlabel("Mass double gammas [GeV]")
plt.ylabel("counts")

In the next step you should find the correct distribution/density function and the *negative log-likelihood function* such that you can later fit the data. The input of your function should be the data points as an array (make sure your function can handle an array as an input!) as a first argument and the parameters you want to fit as a second argument.

In [ ]:
# Density function

## pars[0] = decay constant (lambda) of exponential bg, pars[1] = mu (see exercise sheet, this is NOT the mean position of the peak)

def density(pars,x):
    
    # TODO: 

In [ ]:
# Likelihood

## Negative log-likelihood (bg + Higgs)
### pars[0] = decay constant (lambda) of exponential bg, pars[1] = mu (see exercise sheet, this is NOT the mean position of the peak)

def negloglik(pars,x):

  # First some catches for unphysical (aka nonsense parameter values)
  # If the minimzier chooses such a value we sim_dataurn a high number / a high penalty 
  if pars[0] <= 0:
    return 1e10
  if pars[1] <0: 
    return 1e10
  if pars[1] >1.0:
    return 1e10

  # TODO: write the rest of your negative log-likelihood function

Now you can minimize your function to find the correct fit. Plot your result as the histogram together with the fit, think about the right normalisation of the fit!

In [ ]:
# Minimize for the fit (Note: Nothing to add in this cell)
print(" --- Best fit        ---")
startpars = [0.05, 0.5] # start values for decay constant lambda and mu (see excercise sheet)
minresult_bestfit = scipy.optimize.minimize(negloglik, startpars, args = sim_data, method='BFGS')
print(minresult_bestfit)

# Note: You may ignore a warning like "Desired error not achieved [...]"

In [ ]:
## Plot it (do not forget to shift by x0)
# TODO: find the right function for plotting the fit with the histogram
x_fit = 
y_fit = 

plt.hist(sim_data+x0,bins=nbins,range=(x0,xmax))
plt.plot(x_fit+x0,y_fit,label="Best fit")
plt.xlabel("Mass double gammas [GeV]")
plt.ylabel("counts")
plt.legend()

## Excercise 6.5
_______________

Find the correct likelihood function for the null hypothesis (may reuse something from 6.4).
Then do the Fit.

In [ ]:
def negloglik_null(par,x):
    # TODO    

In [ ]:
print(" --- Null hypothesis ---")
minresult_null = scipy.optimize.minimize(negloglik_null, startpars[0], args = sim_data, method='BFGS')
print(minresult_null)

# Note: You may ignore a warning like "Desired error not achieved [...]"

Now plot again the data together with your best fit for the signal and the Null hypothesis

In [ ]:
# Plot again (data + null hypothese + best fit)

# TODO: find the right function for plotting the Null hypothesis fit with the histogram
x_fit_null = 
y_fit_null = 

plt.hist(sim_data+x0,bins=nbins,range=(x0,xmax))
plt.plot(x_fit+x0,y_fit,label="Best fit signal")
plt.plot(x_fit_null+x0,y_fit_null+x0,label="Best fit Null Hypothesis")
plt.xlabel("Mass double gammas [GeV]")
plt.ylabel("counts")
plt.legend()

### Significance

To calculate the significance, we use [Wilk's Theorem](https://en.wikipedia.org/wiki/Wilks%27_theorem), which states that the test statistic

$$D = -\ln(\Lambda) = -2\ln\left(\frac{\text{Likelihood of the null hypothesis}}{\text{Likelihood of the alternative hypothesis}}\right)=-2\left(\ln(\text{Likelihood of the null hypothesis})-\ln(\text{Likelihood of the alternative hypothesis})\right)$$

approximately follows a $\chi^2$ distribution. The degrees of freedom of the $\chi^2$ distribution are given by the difference in dimensionality between the alternative and null hypotheses (i.e., the size of the free parameter space). In this case, the null hypothesis has one degree of freedom $\lambda$, and the alternative hypothesis has two degrees of freedom $\lambda$ and $\mu$ - so one degree of freedom.

So, first, we calculate the test statistic, i.e., the difference of the values of the two fits (Note: we use the negative log-likelihood so be careful with the signs):



In [ ]:
#TODO calculate and print the test statistic D
D = 
print(D)

We can now examine where this value lies in a $\chi^2$ distribution with one degree of freedom (DOF) and determine the p-value (significance):

In [ ]:
p_value = # TODO: calculate this, use scipy.stats.chi2 and choose the correct function

x_array = np.linspace(3,15,500)
x_fill = np.linspace(D,15,300)
y_array = scipy.stats.chi2.pdf(x_array, df=1)
plt.plot(x_array,y_array,label=r'$\chi^2$ with 1 DOF')
plt.axvline(D, ls = '--', linewidth='1',label='D')
plt.fill_between(x_fill,np.zeros_like(x_fill),scipy.stats.chi2.pdf(x_fill, df=1),color = 'skyblue',alpha=0.8, label='p = {:1.5f}'.format(p_value))
plt.xlim(3,15)
plt.ylim(-0.001,0.01)
plt.xlabel('test statistic D')
plt.ylabel(r'$\chi^2(D)$')
plt.legend()

In particle physics, the significance level is often expressed as a multiple of the standard normal distribution's deviation. From the p-value, the significance Z is obtained through the inverse cumulative distribution function (ppf in Python):

$$Z=\Phi^{-1}(1-p)$$


In [ ]:
def sgnf(p):
    return scipy.stats.norm.ppf(1-p, loc=0, scale=1)

x_array = np.linspace(-5,5,500)
x_fill = np.linspace(sgnf(p_value),5,300)
y_array = scipy.stats.norm.pdf(x_array)
plt.plot(x_array,y_array)
plt.axvline(sgnf(p_value), ls = '--', linewidth='1',label=r'{:1.3f} $\sigma$'.format(sgnf(p_value)))
plt.fill_between(x_fill,np.zeros_like(x_fill),scipy.stats.norm.pdf(x_fill),color = 'skyblue',alpha=0.8, label='p = {:f}'.format(p_value))
plt.xlabel(r'$x$')
plt.ylabel(r'Norm$(x;\mu=0,\sigma=1)$')
plt.legend()
print("Significance =",sgnf(p_value),"sigma")

There is also an approximation for the significance in sigma for the signal test with Log-Likelihood ratio, described in this [paper](https://link.springer.com/content/pdf/10.1140/epjc/s10052-011-1554-0.pdf?pdf=button). According to this approximation, the significance is found quite simply with:

$$Z = \sqrt{D}$$

In [ ]:
# Calculate the significance using the approximation (Note: Nothing to add in this cell)
sig = np.sqrt(D) 
print("Significance approximation =",sig,"sigma")
